# Sentiment Analysis and Stock Prediction Model

## Deep Learning

In [5]:
# import torch
from torch import nn
import math
import pandas as pd

stock = 'MSFT'

stock_data = pd.read_csv(f'stock_data/{stock}_{start_date}_{end_date}.csv')

train_split = int(math.ceil(len(stock_data)*0.8))
test_split = len(stock_data) - train_split

train_data, test_data = stock_data[:train_split].iloc[:,:1], stock_data[train_split:].iloc[:,:1]
print(train_data.shape, test_data.shape)

(853, 1) (213, 1)


In [ ]:
class Model(nn.Module):
    def __init__(self,input_features, output_features,hidden_features):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=hidden_features),
            nn.ReLU(),
            nn.Linear(in_features=hidden_features, output_features=hidden_features),
            nn.ReLU(),
            nn.Linear(in_features=hidden_features, output_features=hidden_features),
            nn.ReLU(),
            nn.Linear(in_features=hidden_features, output_features=output_features),
        )
    def forward(self,x):
        return self.layers(x)
model = Model(input_features=5, output_features=1,hidden_features=16)